In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import librosa
import json

In [ ]:
def create_dataset(folder_path): 
  images = []
  names = []
  for file in os.listdir(folder_path):
    
    image_path = os.path.join(folder_path, file)

    size = Image.open(image_path).size
    image = np.array(Image.open(image_path).resize((int(size[0]/4),int(size[1]/2))))
    image = image.astype('float32')
    image = np.delete(image, [3], axis=2)

    image /= 255.0
    if file[:-4] != 'axqtExFY5-s (1)':
      images.append(image)
      names.append(file[:-4])

  max_len = 0
  for i in range(len(images)):
    length = images[i].shape[1]
    if length>max_len:
      max_len=length

  for i in range(len(images)):
    images[i] = librosa.util.fix_length(images[i], int(max_len), axis=1)

  images = np.array(images)

  df = pd.read_csv('/content/drive/MyDrive/YouTubeID_and_Keywords.csv')

  keywords = []
  for i in names:
    if i in df.YTID.to_list():
      keys = df.Keywords[df.YTID == i].to_list()
      keywords.append(keys[0].split(';'))
    else:
      print(i)

  return names,images,keywords

In [ ]:
names, images, keywords = create_dataset('/content/drive/MyDrive/Pre-Training Images')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
multihot = mlb.fit_transform(keywords)
class_labels = list(mlb.classes_)
print(len(class_labels))

527


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, multihot, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.layers.pooling import AveragePooling2D
from keras.layers.normalization.batch_normalization import BatchNormalization
import tensorflow as tf

In [ ]:
# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(64, 107, 3)))
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(class_labels), activation='softmax'))

In [ ]:
# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.0001, momentum=0.6),
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
295/295 [==============================] - 363s 1s/step - loss: 14.9075 - accuracy: 0.0545 - val_loss: 17.5264 - val_accuracy: 0.0083
Epoch 2/10
295/295 [==============================] - 344s 1s/step - loss: 14.0775 - accuracy: 0.0837 - val_loss: 15.6352 - val_accuracy: 0.0216
Epoch 3/10
295/295 [==============================] - 344s 1s/step - loss: 13.6874 - accuracy: 0.0898 - val_loss: 13.7940 - val_accuracy: 0.0908
Epoch 4/10
295/295 [==============================] - 343s 1s/step - loss: 13.4029 - accuracy: 0.0968 - val_loss: 13.7240 - val_accuracy: 0.1000
Epoch 5/10
295/295 [==============================] - 343s 1s/step - loss: 13.1689 - accuracy: 0.1011 - val_loss: 13.6736 - val_accuracy: 0.1015
Epoch 6/10
295/295 [==============================] - 344s 1s/step - loss: 12.9827 - accuracy: 0.1027 - val_loss: 13.5802 - val_accuracy: 0.0936
Epoch 7/10
295/295 [==============================] - 339s 1s/step - loss: 12.7885 - accuracy: 0.1058 - val_loss: 13.5633 - val_ac